<a href="https://colab.research.google.com/github/GreggHeller1/dendritic_distance/blob/main/scripts/notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#settings
%load_ext autoreload
%autoreload 2
try:
  import google.colab
  in_colab = True
except:
  in_colab = False
print(in_colab)

True


In [ ]:
#installs (for colab only, run this once)
if in_colab:
    ! git clone https://github.com/GreggHeller1/Neuron_Tutorial.git


Cloning into 'Neuron_Tutorial'...
remote: Enumerating objects: 75, done.
remote: Counting objects: 100% (75/75), done.
remote: Compressing objects: 100% (56/56), done.
remote: Total 75 (delta 30), reused 49 (delta 14), pack-reused 0
Unpacking objects: 100% (75/75), done.


In [ ]:
#local imports
#cwd if in colab for imports to work
if in_colab:
    %cd /content/Neuron_Tutorial
    
from src import data_io as io
from src import plotting
from src import computation as comp


/content/Neuron_Tutorial


In [ ]:
#imports
import xarray as xr
import pandas as pd
from matplotlib import pyplot as plt
import os

In [ ]:
#define paths
#cwd if in colab for file loading to work
if in_colab:
    %cd /content/Neuron_Tutorial/scripts
    
test_path = os.path.join('demo_data', 'test.txt')
print(test_path)
print(os.getcwd())
os.path.exists(test_path)

/content/Neuron_Tutorial/scripts
demo_data/test.txt
/content/Neuron_Tutorial/scripts


True

In [ ]:
#data inputs
io.readfile(test_path)



Reading file at demo_data/test.txt
Testing whether we can read the contents of a text file in colab

maybe we need another line?


In [ ]:
#data manipulation


In [ ]:
#plots


In [ ]:
#data output
